In [1]:
# Connecting google colab with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
# Importing all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Reading the 'Melbourne_HousePrice_Data.csv' file as data
data = pd.read_csv('/content/drive/MyDrive/Datasets/Melbourne_HousePrice_Data.csv')
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
# Get the shape of the data frame object
data.shape
# So, 34857 rows and 21 columns

(34857, 21)

In [5]:
# Get the column names
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [6]:
# Check for any missing values
data.isnull().sum()

,0
Suburb,0
Address,0
Rooms,0
Type,0
Price,7610
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1


In [7]:
# Check column types of the data frame object
data.dtypes

,0
Suburb,object
Address,object
Rooms,int64
Type,object
Price,float64
Method,object
SellerG,object
Date,object
Distance,float64
Postcode,float64


In [8]:
# All columns are not useful for this model
# Creating anothe data frame with useful columns
# Store it as data_2
use_cols = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount',
            'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea',
            'Price']
data_2 = data[use_cols]
data_2.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


In [10]:
# Check for Missing values
data_2.isnull().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,3
Distance,1
CouncilArea,3
Bedroom2,8217


In [11]:
# Fill some columns with zero values
cols_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
data_2[cols_zero] = data_2[cols_zero].fillna(0)

In [12]:
# Now check for missing values
data_2.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,0
Distance,0
CouncilArea,3
Bedroom2,0


In [13]:
# Fill the missing values of Landsize, BuildingArea and Price column with their mean values
data_2['Landsize'] = data_2['Landsize'].fillna(data_2['Landsize'].mean())
data_2['BuildingArea'] = data_2['BuildingArea'].fillna(data_2['BuildingArea'].mean())
data_2.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,0
Distance,0
CouncilArea,3
Bedroom2,0


In [14]:
# Drop all the missing values that are not treated
data_2.dropna(inplace = True)
data_2.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,0
Propertycount,0
Distance,0
CouncilArea,0
Bedroom2,0


In [15]:
# Use the get_dummies() method for dealing with categorical variables and creating dummy varibales
data_2 = pd.get_dummies(data_2, drop_first = True)
data_2.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,Suburb_Airport West,Suburb_Albanvale,Suburb_Albert Park,Suburb_Albion,Suburb_Alphington,Suburb_Altona,Suburb_Altona Meadows,Suburb_Altona North,Suburb_Ardeer,Suburb_Armadale,Suburb_Ascot Vale,Suburb_Ashburton,Suburb_Ashwood,Suburb_Aspendale,Suburb_Aspendale Gardens,Suburb_Attwood,Suburb_Avondale Heights,Suburb_Bacchus Marsh,Suburb_Balaclava,Suburb_Balwyn,Suburb_Balwyn North,Suburb_Bayswater,Suburb_Bayswater North,Suburb_Beaconsfield,Suburb_Beaconsfield Upper,Suburb_Beaumaris,Suburb_Bellfield,Suburb_Bentleigh,Suburb_Bentleigh East,Suburb_Berwick,Suburb_Black Rock,Suburb_Blackburn,Suburb_Blackburn North,Suburb_Blackburn South,Suburb_Bonbeach,Suburb_Boronia,Suburb_Botanic Ridge,Suburb_Box Hill,Suburb_Braybrook,Suburb_Briar Hill,...,SellerG_hockingstuart/Buxton,SellerG_hockingstuart/Harcourts,SellerG_hockingstuart/Marshall,SellerG_hockingstuart/Sweeney,SellerG_hockingstuart/Village,SellerG_hockingstuart/hockingstuart,SellerG_iHomes,SellerG_iOne,SellerG_iProperty,SellerG_iSell,SellerG_iTRAK,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria,CouncilArea_Bayside City Council,CouncilArea_Boroondara City Council,CouncilArea_Brimbank City Council,CouncilArea_Cardinia Shire Council,CouncilArea_Casey City Council,CouncilArea_Darebin City Council,CouncilArea_Frankston City Council,CouncilArea_Glen Eira City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Hume City Council,CouncilArea_Kingston City Council,CouncilArea_Knox City Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Manningham City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Maroondah City Council,CouncilArea_Melbourne City Council,CouncilArea_Melton City Council,CouncilArea_Mitchell Shire Council,CouncilArea_Monash City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,4019.0,2.5,2.0,1.0,1.0,202.0,160.2564,1480000.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,2,4019.0,2.5,2.0,1.0,0.0,156.0,79.0000,1035000.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,3,4019.0,2.5,3.0,2.0,0.0,134.0,150.0000,1465000.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,F

In [16]:
# Separate the input features (X) and output features (y)
X = data_2.drop(['Price'], axis = 1)
y = data_2['Price']

In [53]:
# Performing train test split on the data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)

In [54]:
# Define the Linear Regression model object
lr = LinearRegression()

In [55]:
# Fit the model with training data X_train and y_train
lr.fit(X_train, y_train)

LinearRegression()

In [56]:
# Predicting the model outcomes as y_pred
y_pred = lr.predict(X_test)

In [57]:
# Evaluate model performance
print('The cost function of the model is:', r2_score(y_test, y_pred)) # The model can explain 65 % variability of data

The cost function of the model is: 0.1385368316165222


In [58]:
# Evaluating the model of test set
print('The model score on test data is:', lr.score(X_test, y_test))

The model score on test data is: 0.1385368316165222


In [59]:
# Evaluating the model on training data
print('The model score on train data:', lr.score(X_train, y_train))

The model score on train data: 0.6827792395792723


#### As, train sample accuracy is 0.68, but test sample accuracy is 0.13, this is a clear indication of overfitting

In [64]:
# Using Lasso or L1 regularization: Model define, train sample fit and predicting
lasso_lr = Lasso(alpha = 50, max_iter = 100, tol = 0.1)
lasso_lr.fit(X_train, y_train)
y_pred_lasso = lasso_lr.predict(X_test)

In [65]:
# Now Check the train and test sample scores for the lasso regularization model
print('Accuarcy for the train sample:', lasso_lr.score(X_train, y_train))
print('Accuarcy for the test sample:', lasso_lr.score(X_test, y_test))

Accuarcy for the train sample: 0.6766985624766824
Accuarcy for the test sample: 0.6636111369404489


In [69]:
# Performing Ridge and L2 regularization: Model define, train sample fit and predicting
ridge_lr = Ridge(alpha = 50, max_iter = 100, tol = 0.1)
ridge_lr.fit(X_train, y_train)
y_pred_ridge = ridge_lr.predict(X_test)

In [70]:
# Now Check the train and test sample scores for the ridge regularization model
print('Accuarcy for the train sample:', ridge_lr.score(X_train, y_train))
print('Accuarcy for the test sample:', ridge_lr.score(X_test, y_test))

Accuarcy for the train sample: 0.6622376739684328
Accuarcy for the test sample: 0.6670848945194958
